In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
Rx = PolynomialRing(QQ, 'x')
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
R.<x> = PolynomialRing(QQ)
Scplx= PowerSeriesRing(CC,'x',default_prec=300)

def divides(a,b):
    if mod(b,a)==0:return True
    return False

def ord_p_n_nonnegative(p,n):
    if (n in ZZ)==False:return(n,"is not an integer")
    if n==0:return Infinity
    if is_prime(p)==False:return(p," is not prime.")
    if gcd(p,n)==1:
        return 0
    
    for k in [0..n]:
        if divides(p^k,n)==True:
            if divides(p^(k+1),n)==False:
                return k
        
def ord_p(p,n):return ord_p_n_nonnegative(p,abs(n))


def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return xjNew(n,m)/x

def H(n,m): # definition 8.1 (13aug21)
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    power = 1/(m-2)
    frac=numerator/denominator
    return ((S(frac^power)).O(n+1)).polynomial()

def H4(n,m): #per dfmn 8.2 in 2021 Integer paper
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    return ((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()

def polynomialCoefficient(n,poly):
    return poly.list()[n]

def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent

def numericalfactor(poly):#new code for sage 9.6
    pol=D(poly)
    fp=pol.factor()
    lst=[R(factorpairToFactor(factorpair)) for factorpair in fp]
    prd=1
    for k in [0..len(lst)-1]:
        prd=prd*lst[k]
    return(pol/prd)


def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())

def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)

def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})

def deltaDiamond(n,m): #from 'conjecture 9.ipynb' (interpolation paper)
    return D(S(expand((H(n+2,m)^3)/J(n+2,m)))).truncate(n+1)

def deltaDiamondWt12(n,m):
    return D(S(expand((H4(n+2,m)^3)/J(n+2,m)))).truncate(n+1)

def deltaDiamondWt12Strike(n,m): #this is the cusp form from cn 9 of the interpolations paper
    # substitute 2^6*m^3*x for x in def 9.1; not
    # yet defined in draft  21h28jul21
    f=deltaDiamondWt12(n,m)
    fl=f.list()
    df=f.degree()
    sm=sum(fl[k]*(x*m^3*2^6)^k for k in [0..df])
    dsm=sm.degree()
    sml=sm.list()
    notzero=[]
    for entry in sml:
        if (entry!=0):notzero=notzero+[entry]
    leadingterm=notzero[0]
    sm2=sum(fl[k]*(x*m^3*2^6)^k/leadingterm for k in [0..dsm])
    return sm2
        
def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

def first(list):return list[0]
def last(list):return list[-1]

def rmnjntau(n):
    answer=0
    for k in [1..n-1]:
        answer=answer+(6*k-4*(n-k))*sigma(k,3)*sigma(n-k,5)
    answer=(5*sigma(n,3)+7*sigma(n,5))*n/12-35*answer
    return answer

def element(k,listo):
    if k>len(listo):return (print("bad k"))
    return listo[k-1]

def mobiusInverseOfFunction(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        f=f+moebius(dvk)*g(n/dvk)
    return f

def mobiusInverseOfList(g,n):
    dv=divisors(n)
    nd=len(dv)
    f=0
    for k in [1..nd]:
        dvk=element(k,dv)
        gndvk=element(n/dvk,g)
        f=f+moebius(dvk)*gndvk
    return f

def drop_first_element(listo):
    lenth=len(listo)
    answer=[]
    for k in [1..lenth-1]:
        answer=answer+[listo[k]]
    return answer

def factorSeriesFromLists(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
    return nswr

def factorSeriesFromListsWithTracking(lsst,bound):
    truncatedlist=drop_first_element(lsst)
    F=[element(1,truncatedlist)]
    nswr=[-mobiusInverseOfList(F,1)/1]
    for n in [2..bound]:
        summ=0
        for k in [1..n-1]:
            ekf=element(k,F)
            tnk=element(n-k,truncatedlist)
            summ=summ+ekf*tnk
            print((n,k))
        Fn=n*element(n,truncatedlist)-summ
        F=F+[Fn]
        xpn=-mobiusInverseOfList(F,n)/n
        nswr=nswr+[xpn]
        print("answer length: ",len(nswr))
    return nswr



print("ok")

ok


In [2]:
for m in [3..6]:
    print("-----------------------------------------------------------------------")
    print(deltaDiamondWt12(5,m))

-----------------------------------------------------------------------
805/1486016741376*x^5 - 23/80621568*x^4 + 7/82944*x^3 - 1/72*x^2 + x
-----------------------------------------------------------------------
201243/2147483648*x^5 - 17/16384*x^4 + 189/16384*x^3 - 1/8*x^2 + x
-----------------------------------------------------------------------
117919043/655360000000*x^5 - 130141/64000000*x^4 + 13551/640000*x^3 - 37/200*x^2 + x
-----------------------------------------------------------------------
725/2834352*x^5 - 227/78732*x^4 + 37/1296*x^3 - 2/9*x^2 + x


In [3]:
for m in [3..6]:
    print("-----------------------------------------------------------------------")
    print(deltaDiamondWt12Strike(5,m))

-----------------------------------------------------------------------
4830*x^5 - 1472*x^4 + 252*x^3 - 24*x^2 + x
-----------------------------------------------------------------------
26377322496*x^5 - 71303168*x^4 + 193536*x^3 - 512*x^2 + x
-----------------------------------------------------------------------
736994018750*x^5 - 1041128000*x^4 + 1355100*x^3 - 1480*x^2 + x
-----------------------------------------------------------------------
9341553868800*x^5 - 7616856064*x^4 + 5455872*x^3 - 3072*x^2 + x


In [2]:
poly=D(deltaDiamondWt12Strike(200,3))
data=[]
ls=poly.list()
for n in [1..len(ls)-1]:
    data=data+[polynomialCoefficient(n,poly)-rmnjntau(n)]
print(data)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [2]:
import time
import pickle
wfile = open('/Users/barrybrent/run29aug22no5.txt','wb') 
data=[]
start=time.time()
for m in [3..600]:
    ds=deltaDiamondWt12Strike(200,m)
    data=data+[[m,ds]]
    finish=time.time()
    print([m,finish-start])
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 32.3795440196991]
[4, 55.95089912414551]
[5, 98.1343400478363]
[6, 117.78893089294434]
[7, 164.57057809829712]
[8, 200.10892295837402]
[9, 250.88374209403992]
[10, 283.0855529308319]
[11, 334.1815800666809]
[12, 381.4675860404968]
[13, 438.46588802337646]
[14, 478.7286500930786]
[15, 542.3433759212494]
[16, 587.4102659225464]
[17, 650.4979951381683]
[18, 692.1480000019073]
[19, 749.185534954071]
[20, 802.8742480278015]
[21, 867.2701170444489]
[22, 905.862270116806]
[23, 963.6641738414764]
[24, 1016.7312169075012]
[25, 1079.4231610298157]
[26, 1122.6830201148987]
[27, 1186.3621439933777]
[28, 1240.5801479816437]
[29, 1300.9999940395355]
[30, 1348.251667022705]
[31, 1408.4297959804535]
[32, 1458.2197771072388]
[33, 1527.7827541828156]
[34, 1572.7517020702362]
[35, 1644.5583820343018]
[36, 1703.8551969528198]
[37, 1768.850081205368]
[38, 1814.204815864563]
[39, 1884.6672358512878]
[40, 1943.0007140636444]
[41, 2007.7201750278473]
[42, 2059.871999025345]
[43, 2124.5355751514435]
[44, 2

[330, 23940.86260509491]
[331, 24027.47843003273]
[332, 24108.0282330513]
[333, 24203.32035422325]
[334, 24271.945293188095]
[335, 24361.811581134796]
[336, 24447.008177042007]
[337, 24535.330221891403]
[338, 24605.886102199554]
[339, 24699.04035115242]
[340, 24781.299962043762]
[341, 24872.47693014145]
[342, 24945.154958963394]
[343, 25038.671415805817]
[344, 25116.416659116745]
[345, 25213.88414502144]
[346, 25279.487462043762]
[347, 25364.9907579422]
[348, 25450.555145025253]
[349, 25537.971440076828]
[350, 25612.320665836334]
[351, 25706.103350162506]
[352, 25783.81094312668]
[353, 25870.847243070602]
[354, 25943.13269686699]
[355, 26035.949605941772]
[356, 26113.889384031296]
[357, 26212.064081907272]
[358, 26279.06046295166]
[359, 26368.502265930176]
[360, 26455.534321069717]
[361, 26547.62153506279]
[362, 26615.256364822388]
[363, 26712.59976196289]
[364, 26798.36814904213]
[365, 26891.059469223022]
[366, 26963.13778591156]
[367, 27050.95517396927]
[368, 27131.637253046036]
[369

In [3]:
import pickle
rfile =  open('/Users/barrybrent/run29aug22no5.txt','rb') # deltaDiamondWt12Strike srs
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    m = s[k][0]
    poly=D(s[k][1])
    data=data+[[k,poly.degree()]]
    print("k: ",k)
print(data) # correctness check: all of the series have the right degree.

k:  0
k:  1
k:  2
k:  3
k:  4
k:  5
k:  6
k:  7
k:  8
k:  9
k:  10
k:  11
k:  12
k:  13
k:  14
k:  15
k:  16
k:  17
k:  18
k:  19
k:  20
k:  21
k:  22
k:  23
k:  24
k:  25
k:  26
k:  27
k:  28
k:  29
k:  30
k:  31
k:  32
k:  33
k:  34
k:  35
k:  36
k:  37
k:  38
k:  39
k:  40
k:  41
k:  42
k:  43
k:  44
k:  45
k:  46
k:  47
k:  48
k:  49
k:  50
k:  51
k:  52
k:  53
k:  54
k:  55
k:  56
k:  57
k:  58
k:  59
k:  60
k:  61
k:  62
k:  63
k:  64
k:  65
k:  66
k:  67
k:  68
k:  69
k:  70
k:  71
k:  72
k:  73
k:  74
k:  75
k:  76
k:  77
k:  78
k:  79
k:  80
k:  81
k:  82
k:  83
k:  84
k:  85
k:  86
k:  87
k:  88
k:  89
k:  90
k:  91
k:  92
k:  93
k:  94
k:  95
k:  96
k:  97
k:  98
k:  99
k:  100
k:  101
k:  102
k:  103
k:  104
k:  105
k:  106
k:  107
k:  108
k:  109
k:  110
k:  111
k:  112
k:  113
k:  114
k:  115
k:  116
k:  117
k:  118
k:  119
k:  120
k:  121
k:  122
k:  123
k:  124
k:  125
k:  126
k:  127
k:  128
k:  129
k:  130
k:  131
k:  132
k:  133
k:  134
k:  135
k:  136
k:  137
k:  13

In [10]:
R.<x> = PowerSeriesRing(ZZ)
import pickle
rfile =  open('/Users/barrybrent/run29aug22no5.txt','rb') # deltaDiamondWt12Strike srs
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
bound=200
def m(n):return s[n][0]
def pre_g(n):return s[n][1]
def g(n):return list(D(pre_g(n)))
for index in [0..597]:
    print("index: ",index)
    dfeg=drop_first_element(g(index))
    fs=factorSeriesFromLists(dfeg,bound-1)
    data=data+[[m(index),fs]]

wfile = open('/Users/barrybrent/30aug22no1.txt','wb') #deltaDiamondWt12Strike Euler exponents
t = pickle.dumps(str(data))
wfile.write(t)
wfile.close()

index:  0
index:  1
index:  2
index:  3
index:  4
index:  5
index:  6
index:  7
index:  8
index:  9
index:  10
index:  11
index:  12
index:  13
index:  14
index:  15
index:  16
index:  17
index:  18
index:  19
index:  20
index:  21
index:  22
index:  23
index:  24
index:  25
index:  26
index:  27
index:  28
index:  29
index:  30
index:  31
index:  32
index:  33
index:  34
index:  35
index:  36
index:  37
index:  38
index:  39
index:  40
index:  41
index:  42
index:  43
index:  44
index:  45
index:  46
index:  47
index:  48
index:  49
index:  50
index:  51
index:  52
index:  53
index:  54
index:  55
index:  56
index:  57
index:  58
index:  59
index:  60
index:  61
index:  62
index:  63
index:  64
index:  65
index:  66
index:  67
index:  68
index:  69
index:  70
index:  71
index:  72
index:  73
index:  74
index:  75
index:  76
index:  77
index:  78
index:  79
index:  80
index:  81
index:  82
index:  83
index:  84
index:  85
index:  86
index:  87
index:  88
index:  89
index:  90
index:  9

In [11]:
import pickle
rfile = open('/Users/barrybrent/30aug22no1.txt','rb') #deltaDiamondWt12Strike Euler exponents
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
print(len(s))

598


In [12]:
import pickle
rfile = open('/Users/barrybrent/30aug22no1.txt','rb') #deltaDiamondWt12Strike Euler exponents
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
for k in [0..0]:
    print(s[k])

[3, [24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24, 24]]


In [13]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/30aug22no1.txt','rb') #deltaDiamondWt12Strike Euler exponents
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [0..len(s)-1]:
    m = s[k][0]
    exponents=s[k][1]
    data=data+[[k,len(exponents)]]
print(data) 

[[0, 199], [1, 199], [2, 199], [3, 199], [4, 199], [5, 199], [6, 199], [7, 199], [8, 199], [9, 199], [10, 199], [11, 199], [12, 199], [13, 199], [14, 199], [15, 199], [16, 199], [17, 199], [18, 199], [19, 199], [20, 199], [21, 199], [22, 199], [23, 199], [24, 199], [25, 199], [26, 199], [27, 199], [28, 199], [29, 199], [30, 199], [31, 199], [32, 199], [33, 199], [34, 199], [35, 199], [36, 199], [37, 199], [38, 199], [39, 199], [40, 199], [41, 199], [42, 199], [43, 199], [44, 199], [45, 199], [46, 199], [47, 199], [48, 199], [49, 199], [50, 199], [51, 199], [52, 199], [53, 199], [54, 199], [55, 199], [56, 199], [57, 199], [58, 199], [59, 199], [60, 199], [61, 199], [62, 199], [63, 199], [64, 199], [65, 199], [66, 199], [67, 199], [68, 199], [69, 199], [70, 199], [71, 199], [72, 199], [73, 199], [74, 199], [75, 199], [76, 199], [77, 199], [78, 199], [79, 199], [80, 199], [81, 199], [82, 199], [83, 199], [84, 199], [85, 199], [86, 199], [87, 199], [88, 199], [89, 199], [90, 199], [91, 199

In [14]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/30aug22no1.txt','rb') #deltaDiamondWt12Strike Euler exponents
rs=pickle.load(rfile) 
rfile.close()

s=stripQuotationMarks(rs)
polydata=[]
import time

for exponentnumber in [0..1]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        exponents=s[k][1]
        exponent=exponents[exponentnumber]
        data=data+[exponent]
    print("exponent number ",exponentnumber)
    print("data: ",data)

exponent number  0
data:  [24, 512, 1480, 3072, 5432, 8704, 13032, 18560, 25432, 33792, 43784, 55552, 69240, 84992, 102952, 123264, 146072, 171520, 199752, 230912, 265144, 302592, 343400, 387712, 435672, 487424, 543112, 602880, 666872, 735232, 808104, 885632, 967960, 1055232, 1147592, 1245184, 1348152, 1456640, 1570792, 1690752, 1816664, 1948672, 2086920, 2231552, 2382712, 2540544, 2705192, 2876800, 3055512, 3241472, 3434824, 3635712, 3844280, 4060672, 4285032, 4517504, 4758232, 5007360, 5265032, 5531392, 5806584, 6090752, 6384040, 6686592, 6998552, 7320064, 7651272, 7992320, 8343352, 8704512, 9075944, 9457792, 9850200, 10253312, 10667272, 11092224, 11528312, 11975680, 12434472, 12904832, 13386904, 13880832, 14386760, 14904832, 15435192, 15977984, 16533352, 17101440, 17682392, 18276352, 18883464, 19503872, 20137720, 20785152, 21446312, 22121344, 22810392, 23513600, 24231112, 24963072, 25709624, 26470912, 27247080, 28038272, 28844632, 29666304, 30503432, 31356160, 32224632, 33108992, 34

In [18]:
# interpolating polynomials
import pickle
rfile = open('/Users/barrybrent/30aug22no1.txt','rb') #deltaDiamondWt12Strike Euler exponents
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import time
wfile = open('/Users/barrybrent/30aug22no2.txt','wb') #deltaDiamondWt12Strike Euler exponent polynomials
for exponentnumber in [1..199]:
    start = time.time()
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        exponents=s[k][1]
        exponent=exponents[exponentnumber-1]
        data=data+[(m,exponent)]
    rl=Rx.lagrange_polynomial(data)
    polydata=polydata+[[exponentnumber,rl]] 
    finish = time.time()
    print([exponentnumber,finish-start])
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close()

[1, 0.3208808898925781]
[2, 0.37924814224243164]
[3, 0.38561415672302246]
[4, 0.3168468475341797]
[5, 0.3832130432128906]
[6, 0.38706088066101074]
[7, 0.3871879577636719]
[8, 0.3227112293243408]
[9, 0.39175915718078613]
[10, 0.3257327079772949]
[11, 0.381702184677124]
[12, 0.3855288028717041]
[13, 0.38750123977661133]
[14, 0.3245887756347656]
[15, 0.3836982250213623]
[16, 0.4037470817565918]
[17, 0.3378159999847412]
[18, 0.39734578132629395]
[19, 0.4050910472869873]
[20, 0.3355710506439209]
[21, 0.4054999351501465]
[22, 0.34195518493652344]
[23, 0.41687607765197754]
[24, 0.41789722442626953]
[25, 0.35356593132019043]
[26, 0.42819976806640625]
[27, 0.3644599914550781]
[28, 0.43561887741088867]
[29, 0.4530630111694336]
[30, 0.3898029327392578]
[31, 0.46771788597106934]
[32, 0.40299129486083984]
[33, 0.46601319313049316]
[34, 0.4772989749908447]
[35, 0.4910697937011719]
[36, 0.4301590919494629]
[37, 0.5085759162902832]
[38, 0.5297589302062988]
[39, 0.4533681869506836]
[40, 0.5141990184783

In [22]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
for  n in [0..5]:
    print("exponent number: ",n+1)
    print(s[n][1])
    print("====================================================================================")

exponent number:  1
24*x^3 - 48*x^2 - 64*x
exponent number:  2
-12*x^6 + 48*x^5 - 704*x^4 + 1332*x^3 + 2008*x^2 + 32*x
exponent number:  3
32*x^9 - 64*x^8 + 38912/27*x^7 - 12800/3*x^6 + 617984/27*x^5 - 142336/3*x^4 - 1970392/27*x^3 + 16*x^2 + 64/3*x
exponent number:  4
-6*x^12 + 48*x^11 - 41152/9*x^10 + 92096/9*x^9 - 2807872/27*x^8 + 2535680/9*x^7 - 20623198/27*x^6 + 16667432/9*x^5 + 78748448/27*x^4 - 672*x^3 - 992*x^2
exponent number:  5
144/5*x^15 - 288/5*x^14 + 5986688/1125*x^13 - 3661312/225*x^12 + 299741696/675*x^11 - 232659968/225*x^10 + 21124308992/3375*x^9 - 3766673408/225*x^8 + 16841273344/675*x^7 - 17211891712/225*x^6 - 413041197056/3375*x^5 - 24/5*x^3 + 48/5*x^2 + 64/5*x
exponent number:  6
-16*x^18 + 64*x^17 - 33780736/3375*x^16 + 28589824/1125*x^15 - 29316011776/30375*x^14 + 1729624064/675*x^13 - 1082256515072/30375*x^12 + 31909781504/375*x^11 - 3472814206976/10125*x^10 + 212679602672/225*x^9 - 22816987491104/30375*x^8 + 3686528011264/1125*x^7 + 161099555119004/30375*x^6 -

In [23]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    dp=pol.degree()
    print("exponent: ",exponentnumber," degree: ",dp)

exponent:  1  degree:  3
exponent:  2  degree:  6
exponent:  3  degree:  9
exponent:  4  degree:  12
exponent:  5  degree:  15
exponent:  6  degree:  18
exponent:  7  degree:  21
exponent:  8  degree:  24
exponent:  9  degree:  27
exponent:  10  degree:  30
exponent:  11  degree:  33
exponent:  12  degree:  36
exponent:  13  degree:  39
exponent:  14  degree:  42
exponent:  15  degree:  45
exponent:  16  degree:  48
exponent:  17  degree:  51
exponent:  18  degree:  54
exponent:  19  degree:  57
exponent:  20  degree:  60
exponent:  21  degree:  63
exponent:  22  degree:  66
exponent:  23  degree:  69
exponent:  24  degree:  72
exponent:  25  degree:  75
exponent:  26  degree:  78
exponent:  27  degree:  81
exponent:  28  degree:  84
exponent:  29  degree:  87
exponent:  30  degree:  90
exponent:  31  degree:  93
exponent:  32  degree:  96
exponent:  33  degree:  99
exponent:  34  degree:  102
exponent:  35  degree:  105
exponent:  36  degree:  108
exponent:  37  degree:  111
exponent:

In [25]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
for  n in [0..5]:
    print("exponent number: ",n+1)
    pol=s[n][1]
    fp=pol.factor()
    print(fp)
    print("====================================================================================")

exponent number:  1
8*(3*x^2 - 6*x - 8)*x
exponent number:  2
-4*(3*x^5 - 12*x^4 + 176*x^3 - 333*x^2 - 502*x - 8)*x
exponent number:  3
8/27*(108*x^8 - 216*x^7 + 4864*x^6 - 14400*x^5 + 77248*x^4 - 160128*x^3 - 246299*x^2 + 54*x + 72)*x
exponent number:  4
-2/27*(81*x^10 - 648*x^9 + 61728*x^8 - 138144*x^7 + 1403936*x^6 - 3803520*x^5 + 10311599*x^4 - 25001148*x^3 - 39374224*x^2 + 9072*x + 13392)*x^2
exponent number:  5
8/3375*(12150*x^14 - 24300*x^13 + 2245008*x^12 - 6864960*x^11 + 187338560*x^10 - 436237440*x^9 + 2640538624*x^8 - 7062512640*x^7 + 10525795840*x^6 - 32272296960*x^5 - 51630149632*x^4 - 2025*x^2 + 4050*x + 5400)*x
exponent number:  6
-4/30375*(121500*x^17 - 486000*x^16 + 76006656*x^15 - 192981312*x^14 + 7329002944*x^13 - 19458270720*x^12 + 270564128768*x^11 - 646173075456*x^10 + 2604610655232*x^9 - 7177936590180*x^8 + 5704246872776*x^7 - 24884064076032*x^6 - 40274888779751*x^5 + 87025500*x^4 - 181926000*x^3 - 273684375*x^2 + 5082750*x + 81000)*x


In [18]:
def numericalfactor(poly):
    pol=D(poly)
    fp=pol.factor()
    lst=[R(factorpairToFactor(factorpair)) for factorpair in fp]
    prd=1
    for k in [0..len(lst)-1]:
        prd=prd*lst[k]
    return(pol/prd)

import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
for  n in [0..5]:
    print("exponent number: ",n+1)
    poly=D(s[n][1])
    print(poly.factor())
    print(numericalfactor(poly))
    print("====================================================================================")

exponent number:  1
(24) * x * (x^2 - 2*x - 8/3)
24
exponent number:  2
(-12) * x * (x^5 - 4*x^4 + 176/3*x^3 - 111*x^2 - 502/3*x - 8/3)
-12
exponent number:  3
(32) * x * (x^8 - 2*x^7 + 1216/27*x^6 - 400/3*x^5 + 19312/27*x^4 - 4448/3*x^3 - 246299/108*x^2 + 1/2*x + 2/3)
32
exponent number:  4
(-6) * x^2 * (x^10 - 8*x^9 + 20576/27*x^8 - 46048/27*x^7 + 1403936/81*x^6 - 1267840/27*x^5 + 10311599/81*x^4 - 8333716/27*x^3 - 39374224/81*x^2 + 112*x + 496/3)
-6
exponent number:  5
(144/5) * x * (x^14 - 2*x^13 + 374168/2025*x^12 - 228832/405*x^11 + 18733856/1215*x^10 - 14541248/405*x^9 + 1320269312/6075*x^8 - 235417088/405*x^7 + 1052579584/1215*x^6 - 1075743232/405*x^5 - 25815074816/6075*x^4 - 1/6*x^2 + 1/3*x + 4/9)
144/5
exponent number:  6
(-16) * x * (x^17 - 4*x^16 + 2111296/3375*x^15 - 1786864/1125*x^14 + 1832250736/30375*x^13 - 108101504/675*x^12 + 67641032192/30375*x^11 - 1994361344/375*x^10 + 217050887936/10125*x^9 - 13292475167/225*x^8 + 1426061718194/30375*x^7 - 230408000704/1125*x^6 - 

In [21]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
for  n in [0..5]:
    exponentnumber=n+1
    pol1=D(s[n][1])
    pol2=D(s[n][1]-24)
    nf1=numericalfactor(pol1)
    nf2=numericalfactor(pol2)
    print((exponentnumber,nf1,nf2)) #these are leading coefficients, not numerical factors.

(1, 24, 24)
(2, -12, -12)
(3, 32, 32)
(4, -6, -6)
(5, 144/5, 144/5)
(6, -16, -16)


In [23]:
yes=[];no=[]

In [22]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
yes=[]
no=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol1=D(s[n][1])
    pol2=D(s[n][1]-24)
    nf1=numericalfactor(pol1)
    nf2=numericalfactor(pol2)
    if nf1==nf2:yes=yes+[exponentnumber]
    if nf1!=nf2:no=no+[exponentnumber]
    print("n: ",n)
print((len(yes),len(no)))
    

n:  0
n:  1
n:  2
n:  3
n:  4
n:  5
n:  6
n:  7
n:  8
n:  9
n:  10
n:  11
n:  12
n:  13
n:  14
n:  15
n:  16
n:  17
n:  18
n:  19
n:  20
n:  21
n:  22
n:  23
n:  24
n:  25
n:  26
n:  27
n:  28
n:  29
n:  30
n:  31
n:  32
n:  33
n:  34
n:  35
n:  36
n:  37
n:  38
n:  39
n:  40
n:  41
n:  42
n:  43
n:  44
n:  45
n:  46
n:  47
n:  48
n:  49
n:  50
n:  51
n:  52
n:  53
n:  54
n:  55
n:  56
n:  57
n:  58
n:  59
n:  60
n:  61
n:  62
n:  63
n:  64
n:  65
n:  66
n:  67
n:  68
n:  69
n:  70
n:  71
n:  72
n:  73
n:  74
n:  75
n:  76
n:  77
n:  78
n:  79
n:  80
n:  81
n:  82
n:  83
n:  84
n:  85
n:  86
n:  87
n:  88
n:  89
n:  90
n:  91
n:  92
n:  93
n:  94
n:  95
n:  96
n:  97
n:  98
n:  99
n:  100
n:  101
n:  102
n:  103
n:  104
n:  105
n:  106
n:  107
n:  108
n:  109
n:  110
n:  111
n:  112
n:  113
n:  114
n:  115
n:  116
n:  117
n:  118
n:  119
n:  120
n:  121
n:  122
n:  123
n:  124
n:  125
n:  126
n:  127
n:  128
n:  129
n:  130
n:  131
n:  132
n:  133
n:  134
n:  135
n:  136
n:  137
n:  13

In [26]:
numerator(2/3)

2

In [34]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=ord(2,numericalfactor(pol))
    data=data+[nf]
print(data)

TypeError: ord() takes exactly one argument (2 given)

In [49]:
def divides(a,b):
    if mod(b,a)==0:return True
    return False

In [50]:
divides(1,36)

True

In [35]:
def ord_p_n_nonnegative(p,n):
    if (n in ZZ)==False:return(n,"is not an integer")
    if n==0:return Infinity
    if is_prime(p)==False:return(p," is not prime.")
    if gcd(p,n)==1:
        return 0
    
    for k in [0..n]:
        if divides(p^k,n)==True:
            if divides(p^(k+1),n)==False:
                return k
        
def ord_p(p,n):return ord_p_n_nonnegative(p,abs(n))

In [21]:
2 in ZZ

True

In [33]:
abs(-2)

2

In [38]:
ord_p(3,-27)

3

In [39]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(QQ(numericalfactor(pol)))
    od=ord_p(2,nf)
    data=data+[od]
print(data)

[3, 2, 5, 1, 4, 4, 6, 0, 3, 3, 5, 3, 4, 5, 6, 0, 4, 2, 5, 2, 8, 4, 6, 2, 3, 3, 6, 4, 4, 5, 8, 0, 7, 3, 7, 1, 4, 4, 6, 1, 4, 7, 5, 3, 4, 5, 7, 1, 3, 2, 6, 2, 4, 5, 6, 3, 7, 3, 5, 4, 4, 7, 6, 0, 5, 6, 5, 2, 8, 6, 6, 0, 4, 3, 5, 3, 8, 5, 7, 0, 3, 3, 5, 6, 5, 4, 6, 2, 4, 3, 7, 4, 10, 6, 6, 0, 4, 2, 5, 1, 4, 5, 6, 1, 9, 3, 5, 4, 4, 5, 6, 2, 4, 6, 7, 2, 4, 4, 7, 3, 3, 3, 6, 6, 5, 5, 10, 0, 7, 4, 5, 5, 8, 4, 7, 1, 4, 7, 5, 5, 9, 5, 6, 0, 5, 3, 5, 2, 4, 4, 6, 2, 4, 7, 9, 4, 4, 6, 6, 0, 9, 2, 5, 2, 8, 4, 6, 5, 3, 4, 5, 3, 4, 5, 6, 1, 7, 3, 5, 2, 4, 6, 6, 3, 5, 9, 6, 5, 9, 5, 9, 0, 4, 3, 7, 1, 4, 4, 6]


In [40]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(QQ(numericalfactor(pol)))
    od=ord_p(3,nf)
    data=data+[od]
print(data)

[1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 0, 2, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 3, 2, 0, 1, 1, 0, 2, 2, 0, 1, 2, 0, 2, 2, 0, 2, 1, 2, 1, 4, 0, 3, 1, 0, 2, 2, 1, 1, 1, 0, 1, 2, 1, 1, 3, 1, 2, 3, 0, 1, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 0, 4, 1, 1, 2, 3, 0, 1, 2, 0, 2, 2, 0, 1, 2, 0, 1, 2, 2, 1, 1, 1, 4, 4, 0, 1, 3, 0, 1, 2, 0, 3, 2, 0, 2, 3, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0, 2, 2, 1, 1, 1, 0, 3, 2, 1, 1, 2, 1, 3, 2, 0, 3, 1, 1, 1, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 3, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 4, 0, 1, 2, 1, 1, 2, 1, 3, 3, 0, 1, 1, 0, 2, 2, 0, 4, 2, 0, 2, 2, 0, 1, 1, 1, 2, 3, 0, 1]


In [43]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(QQ(numericalfactor(pol)))/rmnjntau(exponentnumber)
    data=data+[numerator(nf)]
print(data)

[24, 1, 8, 3, 24, 1, -24, -1, -104, 1, 24, 1, -24, -1, 8, -3, -24, -13, 24, 3, -8, 1, 24, -1, -744, -1, -8, -3, 24, 1, -24, 1, 8, -1, -24, -13, -456, 1, -8, -1, 24, -1, -264, 3, -104, 1, 24, -1, -456, -31, -8, -3, -24, -1, 24, 1, 8, 1, -24, 1, 744, -1, 8, -3, -24, 1, -408, -3, 8, -1, 24, 13, 888, -19, -248, 3, -24, -1, 24, -3, 968, 1, -24, -1, -24, -11, 8, -1, -24, -13, 24, 3, -8, 1, 24, 1, 24, -19, -8, -93, 408, -1, -312, 1, -8, -1, 24, -1, 264, 1, -152, 3, -456, 1, 24, 3, 8, -1, 24, -1, 3192, 31, 8, -3, -312, 1, -24, -1, -88, -1, 264, 1, -24, -17, -8, 1, -24, 1, 24, -3, 8, 1, -24, 13, 24, 37, -152, -57, -24, -31, -456, -1, 104, -1, -24, -1, 1896, 1, -8, 1, -24, 121, -984, 3, 8, -1, 24, 1, -1464, -1, -104, -33, -696, 1, 744, -3, -8, -1, 24, -13, -312, 1, 248, -1, -456, -1, -24, 3, 8, 1, 24, -1, 2328, 1, -8, -57, -264, -1, 24]


In [45]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(numerator(QQ(numericalfactor(pol)))/rmnjntau(exponentnumber))
    if abs(nf)==24:
        data=data+[exponentnumber]
print(data)

[1, 5, 7, 11, 13, 17, 19, 23, 29, 31, 35, 41, 47, 53, 55, 59, 65, 71, 77, 79, 83, 85, 89, 91, 95, 97, 107, 115, 119, 127, 133, 137, 139, 143, 145, 149, 155, 161, 167, 179, 187, 191, 199]


In [47]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    if is_prime(exponentnumber)==True:
        pol=D(s[n][1])
        nf=numerator(numerator(QQ(numericalfactor(pol)))/rmnjntau(exponentnumber))
        if abs(nf)!=24:
            data=data+[exponentnumber]
print(data)

[2, 3, 37, 43, 61, 67, 73, 101, 103, 109, 113, 131, 151, 157, 163, 173, 181, 193, 197]


In [48]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(QQ(numericalfactor(pol)))
    data=data+[numerator(nf)]
print(data)

[24, -12, 32, -6, 144, -16, 192, -3, 104, -72, 288, -8, 336, -96, 192, -3, 432, -52, 480, -36, 256, -144, 576, -4, 744, -168, 320, -48, 720, -96, 768, -3, 384, -216, 1152, -26, 912, -240, 448, -18, 1008, -128, 1056, -72, 208, -288, 1152, -2, 1368, -372, 576, -84, 1296, -160, 1728, -24, 640, -360, 1440, -48, 1488, -384, 832, -3, 2016, -192, 1632, -108, 768, -576, 1728, -13, 1776, -456, 992, -120, 2304, -224, 1920, -9, 968, -504, 2016, -64, 2592, -528, 960, -36, 2160, -104, 384, -144, 1024, -576, 576, -1, 2352, -684, 416, -186, 2448, -288, 2496, -42, 1536, -648, 2592, -80, 2640, -864, 1216, -12, 2736, -320, 3456, -180, 112, -720, 3456, -24, 3192, -744, 1344, -192, 3744, -416, 3072, -3, 1408, -1008, 3168, -96, 3840, -816, 128, -54, 3312, -384, 3360, -288, 1536, -864, 4032, -13, 864, -888, 1824, -228, 3600, -496, 3648, -60, 624, -1152, 4608, -112, 3792, -960, 1728, -9, 4608, -484, 3936, -252, 2304, -1008, 4032, -32, 4392, -1296, 2080, -264, 4176, -480, 5952, -18, 1920, -1080, 4320, -52, 43

In [50]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(QQ(numericalfactor(pol)))
    data=data+[ord_p(2,numerator(nf))]
print(data)

[3, 2, 5, 1, 4, 4, 6, 0, 3, 3, 5, 3, 4, 5, 6, 0, 4, 2, 5, 2, 8, 4, 6, 2, 3, 3, 6, 4, 4, 5, 8, 0, 7, 3, 7, 1, 4, 4, 6, 1, 4, 7, 5, 3, 4, 5, 7, 1, 3, 2, 6, 2, 4, 5, 6, 3, 7, 3, 5, 4, 4, 7, 6, 0, 5, 6, 5, 2, 8, 6, 6, 0, 4, 3, 5, 3, 8, 5, 7, 0, 3, 3, 5, 6, 5, 4, 6, 2, 4, 3, 7, 4, 10, 6, 6, 0, 4, 2, 5, 1, 4, 5, 6, 1, 9, 3, 5, 4, 4, 5, 6, 2, 4, 6, 7, 2, 4, 4, 7, 3, 3, 3, 6, 6, 5, 5, 10, 0, 7, 4, 5, 5, 8, 4, 7, 1, 4, 7, 5, 5, 9, 5, 6, 0, 5, 3, 5, 2, 4, 4, 6, 2, 4, 7, 9, 4, 4, 6, 6, 0, 9, 2, 5, 2, 8, 4, 6, 5, 3, 4, 5, 3, 4, 5, 6, 1, 7, 3, 5, 2, 4, 6, 6, 3, 5, 9, 6, 5, 9, 5, 9, 0, 4, 3, 7, 1, 4, 4, 6]


In [51]:
import pickle
rfile = open('/Users/barrybrent/30aug22no2.txt','rb') #deltaDiamondWt12Strike Euler exponent polynomials
rs=pickle.load(rfile) 
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for  n in [0..len(s)-1]:
    exponentnumber=n+1
    pol=D(s[n][1])
    nf=numerator(QQ(numericalfactor(pol)))
    data=data+[ord_p(3,numerator(nf))]
print(data)

[1, 1, 0, 1, 2, 0, 1, 1, 0, 2, 2, 0, 1, 1, 1, 1, 3, 0, 1, 2, 0, 2, 2, 0, 1, 1, 0, 1, 2, 1, 1, 1, 1, 3, 2, 0, 1, 1, 0, 2, 2, 0, 1, 2, 0, 2, 2, 0, 2, 1, 2, 1, 4, 0, 3, 1, 0, 2, 2, 1, 1, 1, 0, 1, 2, 1, 1, 3, 1, 2, 3, 0, 1, 1, 0, 1, 2, 0, 1, 2, 0, 2, 2, 0, 4, 1, 1, 2, 3, 0, 1, 2, 0, 2, 2, 0, 1, 2, 0, 1, 2, 2, 1, 1, 1, 4, 4, 0, 1, 3, 0, 1, 2, 0, 3, 2, 0, 2, 3, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0, 2, 2, 1, 1, 1, 0, 3, 2, 1, 1, 2, 1, 3, 2, 0, 3, 1, 1, 1, 2, 0, 1, 1, 1, 2, 2, 0, 1, 1, 3, 2, 2, 0, 1, 2, 2, 2, 2, 0, 2, 4, 0, 1, 2, 1, 1, 2, 1, 3, 3, 0, 1, 1, 0, 2, 2, 0, 4, 2, 0, 2, 2, 0, 1, 1, 1, 2, 3, 0, 1]
